In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pandas as pd 
from scipy import optimize
from scipy import integrate
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import dash
dash.__version__
from dash import dcc as dcc
from dash import html as html
from dash.dependencies import Input, Output,State
import plotly.graph_objects as go
import os


sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 50)

### Data set cleaned for Covid-19 modelling
<p>Goal of the exercise a dynamic dashboard, where one can select the fit for different countries.</p>

<p>Fit of SIR model for 5 countries Germany, Italy, Spain, US and South Korea. </p>

In [2]:
#data loading
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';')  
df_analyse = df_analyse.sort_values('date',ascending=True)
df_analyse

,date,Italy,US,Spain,Germany,"Korea, South"
0,2020-01-22,0,1,0,0,1
1,2020-01-23,0,1,0,0,1
2,2020-01-24,0,2,0,0,2
3,2020-01-25,0,2,0,0,2
4,2020-01-26,0,5,0,0,3
...,...,...,...,...,...,...
913,2022-07-23,20608190,90398709,13204863,30331133,19211613
914,2022-07-24,20660065,90410386,13204863,30331133,19247496
915,2022-07-25,20684182,90567290,13204863,30476605,19346764
916,2022-07-26,20772833,90733888,13203228,30598385,19446946


### Fitting the parameters of SIR model
the SIR model is assuming a very simplistic curve however we can find situations (time windows) where the model might apply

In [3]:
# first generate simulated data (number of infected people) using beta and gamma values (default value) using SIR_model_t.
def SIR_model_t(SIR,t,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

In [4]:
#second step: find hyperparameters (beta and gamma) parameters by finding a function (using optimize.curve). Here fit_odeint func 
#outputs the integration values for S, I, R variable. optimize.curve uses simulated y data and x data to find the estimated function 
#(by finialising the hyperparameter values). And finally fitting that estimated function on the simulated data function (using fit_odeint). 

# the resulting curve has to be fitted
# free parameters are here beta and gamma
def fit_odeint(x, beta, gamma):
    '''helper function for the integration.'''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI

In [11]:
print(df_analyse.columns)
c_list = df_analyse.columns[1:]

SIR_df = pd.DataFrame()
#N0=10000000  # total population of Germany country
#approximately 1/8th of original population of each country is considered.

N0_dict = {}
N0_dict['Italy'] = 7000000        # original 59M - 7M
N0_dict['US'] = 41000000         #original 329M - 41M
N0_dict['Spain'] = 6000000      #original 47M - 6M
N0_dict['Germany'] = 10000000    #original 80M  - 10M
N0_dict['Korea, South'] = 7000000   #original 51M  - 7M

for country in c_list:
    ydata = np.array(df_analyse[country][40:150])
    t=np.arange(len(ydata))    
    N0 = N0_dict[country]
    I0=ydata[0]
    S0=N0-I0
    R0=0
    print('start infected:',I0)
    print('cumulative sum of invected after period',ydata[-1])
    print('Number of days',len(ydata))
    print('N0',N0)
    
    col_name = country+'_orig'
    print(col_name)
    SIR_df[col_name] = ydata
    popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)
    perr = np.sqrt(np.diag(pcov))
    
    print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
    print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])

    # get the final fitted curve / predict the outcome 
    fitted=fit_odeint(t, *popt)
    col_name = country+'_SIR'
    print(col_name)
    SIR_df[col_name] = fitted
    print("------------------------------")

print(df_analyse['date'][40:150])    
SIR_df['date'] = np.array(df_analyse['date'][40:150])
print(SIR_df['date'])
SIR_df.to_csv('../data/processed/SIR_df.csv')

Index(['date', 'Italy', 'US', 'Spain', 'Germany', 'Korea, South'], dtype='object')
start infected: 2036
cumulative sum of invected after period 238011
Number of days 110
N0 7000000
Italy_orig
standard deviation errors :  [0.00557382 0.00502851]  start infect: 2036
Optimal parameters: beta = 0.2938688935413914  and gamma =  0.21400505625988717
Italy_SIR
------------------------------
start infected: 55
cumulative sum of invected after period 2219508
Number of days 110
N0 41000000
US_orig


C:\Users\ANSHUG~1\AppData\Local\Temp/ipykernel_2860/502936085.py:17: RuntimeWarning: overflow encountered in double_scalars
  dS_dt=-beta*S*I/N0          #S*I is the
C:\Users\ANSHUG~1\AppData\Local\Temp/ipykernel_2860/502936085.py:18: RuntimeWarning: overflow encountered in double_scalars
  dI_dt=beta*S*I/N0-gamma*I
C:\Users\ANSHUG~1\AppData\Local\Temp/ipykernel_2860/502936085.py:19: RuntimeWarning: overflow encountered in double_scalars
  dR_dt=gamma*I


standard deviation errors :  [0.00903643 0.00893226]  start infect: 55
Optimal parameters: beta = 0.41304807205277827  and gamma =  0.28040631052452203
US_SIR
------------------------------
start infected: 120
cumulative sum of invected after period 245575
Number of days 110
N0 6000000
Spain_orig
standard deviation errors :  [0.00595283 0.00609117]  start infect: 120
Optimal parameters: beta = 0.37063115725075263  and gamma =  0.25671440521373795
Spain_SIR
------------------------------
start infected: 150
cumulative sum of invected after period 188534
Number of days 110
N0 10000000
Germany_orig
standard deviation errors :  [0.01236702 0.0121435 ]  start infect: 150
Optimal parameters: beta = 0.502585456464543  and gamma =  0.39714281267326945
Germany_SIR
------------------------------
start infected: 4335
cumulative sum of invected after period 12373
Number of days 110
N0 7000000
Korea, South_orig
standard deviation errors :  [0.00781522 0.00651962]  start infect: 4335
Optimal paramet

### Visual Dashboard for SIR data modelling

In [12]:
c_list = df_analyse.columns[1:]
print(c_list)
fig = go.Figure()
app = dash.Dash()
app.layout = html.Div([
    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data - SIR virus spread modelling'''),
    
    dcc.Markdown('''
    ## Country for SIR-model visualization (for 110 days)
    '''),
    
    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': country,'value':country} for country in c_list],
        value='Germany', # pre-selected
        multi=False
    ),
    
    dcc.Graph(figure=fig, id='sir_graph_update')
])
    
@app.callback(
    Output('sir_graph_update', 'figure'),
    [Input('country_drop_down', 'value')])
def update_figure(country):
    traces = []
    #for country in c_list:
    col_name = country+'_orig'
    #df_plot = SIR_df[col_name]
    df_plot = np.ediff1d(SIR_df[col_name], to_begin=SIR_df[col_name][1]-SIR_df[col_name][0])
    t=SIR_df['date'] 
    col_name = country+'_SIR'
    df_sir_plot = SIR_df[col_name]
        
    traces.append(
        dict(
            x = t,
            y = df_plot,
            mode = 'markers',
            opacity = 0.9,
            name = country+' Original infected population'
        )
    )

    traces.append(
        dict(
            x = t,
            y = df_sir_plot,
            mode = 'lines',
            opacity = 0.9,
            name = country+' SIR modelled infected population'
        )
    )
    
    return {
        'data': traces,
        'layout': dict(
            width = 1280,
            height = 720,
            xaxis={
                'title': 'Number of Days'
            },
            yaxis = {
                'type':"log",
                'title': 'New Infected Population'
            }
        )
    }
    
    

Index(['Italy', 'US', 'Spain', 'Germany', 'Korea, South'], dtype='object')


In [13]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
